# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

# New Section

In [0]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2020-05-25 23:31:22--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:863:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg.2’

La_noche_estrellada 100%[===================>] 218.48K   850KB/s    in 0.3s    

2020-05-25 23:31:22 (850 KB/s) - ‘La_noche_estrellada1.jpg.2’ saved [223725/223725]

--2020-05-25 23:31:25--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:863:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153015 (149K) [

In [0]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

In [0]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("/content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path("/content/La_noche_estrellada1.jpg")
result_prefix = Path("/content/output")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:

**Total variation weight**: es un peso que se usa como entrada en la total variation loss, que permite mirar una relación entre los píxeles que están próximos / cerca uno del otro de la imagen generada. Permite suavizar la imagen final.

**Style weight**: es un peso que se usa como entrada en la style loss, está relacionado con la paleta de colores, patrones, y texturas que contienen la imagen de referencia mientras que la imagen de contenido no es alterada.

**Content weigtht**: es un peso que usa como entrada en la content loss, y está relacionado con la estructura general y los componentes de alto nivel de la imagen objetivo original.




In [0]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [0]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:

**load_img:** lo que hace es cargar una imagen que está definida en el path "target_path". 
El parámetro "target_size" permite que una tupla (alto y ancho - img_nrows, img_ncols) se setee, redefiniendo la dimensión de la imagen automáticamente luego de que es cargada. La imagen se devuelve en la variable img.

**img_to_array:** lo que hace es agregar canales a "img": 3 canales para una imagen RGB y 1 canal para una imagen gris. Ejemplo: 
img.shape = (img_rows, img_cols, 3) => para RGB

img.shape = (img_rows, img_cols, 1) => para gris (blanco/negro)

La imagen resultante se devuelve en img.

**np.expand_dims:** lo que hace es agregar un número de imágenes.
Agranda el shape de un array, insertando un nuevo eje (dimensión) que aparecerá en la posición del eje en la forma de matriz expandida.
El parámetro axis = 0, lo que hace es posicionar la imagen en un nuevo eje.
La imagen resultante se devuelve en img.

**process_input:** lo que hace es adecuar la imagen al formato que necesita el modelo. Algunos modelos utilizan imágenes con valores del 0 al 1, otros modelos del -1 al +1. Para esto, se deberían cargar imágenes que son compatibles con las funciones de Keras, así nos aseguramos que las imágenes que cargamos con compatibles con la función process_input.
La imagen resultante se devuelve en img.






In [0]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

**Respuesta:**

Lo que lo hace la celda es el proceso inverso a la función anterior, regenerando una imagen.

Lo que hace en la función deprocess_image es: darle un nuevo shape a la imagen x,  en este caso el shape de una tupla del tipo (img_rows, img_ncols, 3).
Luego lo que se hace es lo contrario a lo que se realizó en el paso de preprocess_input: con cada una de las tuplas se vuelven a sumar los valores 103.939, 116.779 y 123.68 (que son las medias de los canales BGR).
Como último paso, lo que se hace es almacenar en x valores enteros sin signo de 8 bits que estén en el rango del 0 al 255. 

In [0]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [0]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [0]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [0]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [0]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?

**Respuesta:**
La matriz de Gram es considerada una métrica.
Para la style loss, se usa la matriz de Gram de la activación de una capa, que calcula el producto interno de los features de una capa.
Este producto es la representación de un mapa de correlaciones entre los features de una capa.
Los términos del producto interno son proporcionales a las covarianzas de 
los features correspondientes, por lo tanto detecta patrones de correlación entre los features de una capa que se activan juntos. 
Estas correlaciones obtienen las estadísticas de patrones que se dan en una determinada escala espacial, que corresponden al estilo, textura, y apariencia y no los componentes y objetos que se encuentran en una imagen.
Si el resultado de la correlación es alto, el resultado de la multiplicación es alto, entonces nosotros podemos decir que esas dos activaciones son correlacionados, de lo contrario, si son diferentes y la multiplicación tiene un resultado bajo, entonces no están correlacionados. 


- ¿Por qué se permutan las dimensiones de x?

**Respuesta:**
Esto se hace para poder multiplicar la matriz, ya que que la matriz de Gram se define como el producto punto de la matriz de estilo con su propia transpuesta.



In [0]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

**Rta:**

**Style loss:**

La style loss se define como la norma de la diferencia entre las matrices de Gram del estilo de referencia y las imágenes generadas.
Cuando se minimiza la style loss, nos aseguramos que las texturas encontradas en escalas espaciales diferentes en la imagen de estilo de referencia serán muy parecidas en la imagen generada.

Para poder capturar  el estilo de una imagen podemos obtener la representación mirando la correlación de los valores entre las activaciones o las características.

**Content loss:**
Se define como la diferencia cuadrática media entre lo que se denomina contenido y target en la función Loss.
Si asumimos que normalmente obtenemos los features de representaciones relevantes del contenido de las imágenes de las capas superiores de una CNN, se espera que la imagen que se genera sea muy similar a la a la imagen objetivo original.
Mientras que el contenido se puede definir como la estructura general y los componentes de alto nivel de la imagen objetivo original.

**Total variation loss:**
Total variation loss es similar a la loss de regularización. Se utiliza para garantizar una continuidad espacial y a su vez suavidad en la imagen generada para evitar tener resultados ruidosos y demasiado pixelados.
Opera en los pixeles de la imagen combinada generada.




In [0]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [0]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [0]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [0]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [0]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

**Respuesta:**
Lo que hace la función fmin_l_bfgs_b es minimizar una función usando el algoritmo L-BFGS-B.
En este caso se ejecuta la optimización sobre los pixeles de la imagen generada para minimizar la style loss.

Se utiliza la clase Evaluator para que se calculen los valores del gradiente y de la loss.
Durante la ejecución de la celda donde se encuentra la función fmin_l_bfgs_b, se evalúa la loss y se guarda cada imagen generada en cada una de las iteraciones. En total se ejecutan 100 iteraciones. Si se observa cada una de las imágenes, se puede ver como va cambiando el estilo de cada una en cada iteración que se ejecuta.

En esta notebook lo que se implementa es la función fmin_l_bfgs_b para la optimización de la loss, en cambio lo que se expone en el paper es la utilización de back propagation con gradient descent.

Para minimizar la loss se puede utilizar el algoritmo l-bfgs o bien se podría también usar ADAM. Actualizamos iterativamente nuestra imagen de salida para poder minimizar la loss. No actualizamos los pesos asociados con nuestra red sino que en lugar de eso entrenamos nuestra imagen de entrada para poder minimizar la loss.


In [0]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [0]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [0]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 13289544000.0
Image saved as /content/output/output_at_iteration_0.png
Iteration 0 completed in 3s
Start of iteration 1
Current loss value: 6094290000.0
Image saved as /content/output/output_at_iteration_1.png
Iteration 1 completed in 3s
Start of iteration 2
Current loss value: 4249901000.0
Image saved as /content/output/output_at_iteration_2.png
Iteration 2 completed in 3s
Start of iteration 3
Current loss value: 3348618800.0
Image saved as /content/output/output_at_iteration_3.png
Iteration 3 completed in 3s
Start of iteration 4
Current loss value: 2715652000.0
Image saved as /content/output/output_at_iteration_4.png
Iteration 4 completed in 3s
Start of iteration 5
Current loss value: 2328197000.0
Image saved as /content/output/output_at_iteration_5.png
Iteration 5 completed in 3s
Start of iteration 6
Current loss value: 2099566100.0
Image saved as /content/output/output_at_iteration_6.png
Iteration 6 completed in 3s
Start of iteration 7
Curre

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

**Respuesta:**
Habiendo hecho ejecuciones con distintos pesos en total variation weight, style weight y content weight podría decir que:
- Para style weight: si se aumenta más este peso, en la imagen final se percibe más la imagen de estilo que la de contenido.
- Para content weight: si se aumenta más este peso, en la imagen final se percibe más la imagen de contenido (imagen original) que el estilo que se le quiere dar.
- Para total variation weight: ayuda a suavizar la imagen final. Pero si se aumenta demasiado este peso, la imagen final tiene mucho ruido, es decir que la calidad se va perdiendo ya que está demasiado "suavizada" y los colores de que hay en la imagen original resultan en la imagen final entremezclados.
Si dejo todos los pesos con valores en 0, la imagen resultante termina siendo igual a la imagen de contenido.

Adjunto las imágenes generadas.

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

**Respuesta:**
Cambié las imágenes de contenido y de estilo, y volví a ejecutar. 
Adjunto en una carpeta los archivos (imágenes) que se fueron generando y la imagen resultante.